<a href="https://colab.research.google.com/github/Ang3lino/recomenderSys/blob/master/collaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
import pandas as pd

import os
import pickle

from sortedcontainers import SortedList
from collections import Counter, defaultdict
from tqdm import tqdm  # modulo cuya finalidad es dar un feedback del progreso de algun procedimiento

In [3]:

!pip install tqdm

# User-Based Neighborhood Models
En este enfoque, user-based neighborhoods son definidos con el fin de identificar usuario similares a el usuario objetivo con el fin de predecir valoraciones que el daria a un articulo.

Para la matrix de valoraciones $m \times n$, $R = [r_{ui}]$ con $m$ usuarios y $n$ articulos. Definamos $I_u$ el conjunto de indices de articulos que el usuario $u$ ha valorado (una fila). El conjunto $I_u \cap I_v$ define las observaciones mutuamente observadas.

In [4]:
from google.colab import drive  
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_object(fname: str, user_count: int, item_count: int) -> defaultdict:
    fdir = 'drive/My Drive/petroleo/movielens-20m-dataset'
    fname = f'{fname}_{user_count}_{item_count}.json'
    fpath = os.path.join(fdir, 'shrinked', fname)
    with open(fpath, 'rb') as fp:
        object_ = pickle.load(fp)
    return object_

def defaultdict_set(defdict):
    return {k: set(v) for k, v in defdict.items()}


user_count = 4096
item_count = 512
user2item = load_object('user2item', user_count, item_count)
item2user = load_object('item2user', user_count, item_count)
user_item2rating = load_object('user_item2rating', user_count, item_count)

user2item = defaultdict_set(user2item)

# Pearson
Una medida que captura la similitud $Sim(u, v)$ entre los vectores de valoracion entre usuarios $u,v$ es el coeficiente de correlacion de Pearson. El primer paso es calcular las medias de valoracion $\mu_u$ para cada usuario $u$:

$$ \mu_u = \frac{\sum_{k\in I_u}r_{uk}}{|I_u|} $$

Asi, la correlacion de Parson se define como 

$$ Sim(u, v) = Pearson(u, v) = \frac{\sum_{k \in I_u \cap I_v}(r_{uk} - \mu_u)(r_{vk} - \mu_v)}
{\sqrt{\sum_{k \in I_u \cap I_v}(r_{uk} - \mu_u)^2 }  \sqrt{ \sum_{k \in I_u \cap I_v}(r_{vk} - \mu_v)^2 } 
} $$



In [0]:
def user_means(user_item2rating: dict, user2item: defaultdict) -> dict:
    means = dict() 
    for u, items in user2item.items():
        means[u] = np.mean([user_item2rating[(u, i)] for i in items])
    return means

def sim_cos(u: np.array, v: np.array):
    dot = lambda u, v: u.dot(v)
    sigma = lambda u, v: np.sqrt(dot(u, v))
    num = dot(u, v)
    den = sigma(u, u) * sigma(v, v)
    return num / den
    

def sim_pearson(u: int, v: int, means: dict, user2item: defaultdict, user_item2rating: dict):
    intersection = user2item[u] & user2item[v]
    a = np.array([user_item2rating[u, k] for k in intersection]) - means[u]
    b = np.array([user_item2rating[v, k] for k in intersection]) - means[v]
    return sim_cos(a, b)



In [7]:
means = user_means(user_item2rating, user2item)
ans = sim_pearson(1, 5, means, user2item, user_item2rating)
print(ans)

0.11996713555118083


# Prediccion
Sea $P_u(j)$ el conjunto de los k usuarios mas cercanos del usuario $u$, quienes han valorado el articulo $j$. Usuarios con un muy bajo o negativas correlaciones con el usuario $u$ son filtrados por $P_u(j)$ como una mejora heuristica. Asi, la funcion de prediccion se describe como: 

$$\hat{r_{uj}} = \mu_u + \frac{\sum_{v\in P_u(j)}Sim(u,v)(r_{vj}-\mu_v)}{\sum_{v\in P_u(j)}|Sim(u, v)|} $$

In [0]:
def euclidean_distance(u, v):
    return np.sqrt(np.sum((u - v) ** 2))

def neighbors(x_vec, y_cls, p_i, k=3):
    dist_class = [(euclidean_distance(x_i, p_i), y_cls[i]) for i, x_i in enumerate(x_vec)]
    sorted_distance = sorted(dist_class, key=lambda x: x[0])[:k]
    return [y_i for (_, y_i) in sorted_distance]

def closest_users(u: int, j: int, item2user, user_item2rating):
    users = item2user[j]
    ratings = [user_item2rating[user, j] for user in users]
    closests = neighbors(ratings, users, user_item2rating[(u, j)])
    return closests


In [9]:
u = 1
j = 2
closest_users(u, j, item2user, user_item2rating)

[405.0, 691.0, 1005.0]

In [0]:
def predict_rating(u: int, j: int, means: dict, user2item, item2user, user_item2rating):
    k_closest = closest_users(u, j, item2user, user_item2rating)
    similars = np.array([sim_pearson(u, v, means, user2item, user_item2rating) for v in k_closest])
    s_vj = np.array([user_item2rating[(v, j)] - means[v] for v in k_closest])
    num = np.sum(similars * s_vj ) 
    den = np.sum(list(map(abs, similars)))
    return means[u] + num/den

In [11]:
r_pred = predict_rating(u, j, means, user2item, item2user, user_item2rating)
print(r_pred)
print(user_item2rating[(u, j)])

4.629527360969084
4.0


In [0]:
def predict(user2item, item2user, user_item2rating, batch_size: int) -> dict:
    predictions = dict()
    # n = len(user2item)
    count = 0
    for u, items in tqdm(user2item.items(), total=batch_size):
        for i in items:
            predictions[(u, i)] = predict_rating(u, i, means, user2item, item2user, user_item2rating)
        count += 1
        if count > batch_size: break
    return predictions

def mse(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred) ** 2))

In [13]:
%time preds = predict(user2item, item2user, user_item2rating, 100)

100%|██████████| 100/100 [11:27<00:00,  7.22s/it]

CPU times: user 11min 36s, sys: 197 ms, total: 11min 36s
Wall time: 11min 37s


In [14]:
debug = True
acc = 0
for u, i in preds.keys():
    x = preds[u, i] 
    y = user_item2rating[u, i]
    if debug: print(x, y)
    acc += (x - y) ** 2 
print(np.sqrt(acc))

Streaming output truncated to the last 5000 lines.
4.319727279478671 4.5
3.770071893357829 3.5
4.166400299561167 4.0
3.12228346411513 2.5
3.4530102711584725 3.5
3.712438059553548 4.0
3.383374975685777 3.5
2.570046784085568 2.5
4.49140233545128 4.5
3.8431911793167197 4.0
4.86110114930971 4.5
3.4630780029570065 3.5
3.1195744454307444 3.0
3.3298349610997873 3.0
2.8211345624837003 2.5
4.25693953250749 4.0
3.7631293596280284 3.5
3.676812511396614 3.5
4.034578384673234 4.0
4.483500608637904 4.5
3.545340459674516 3.5
4.712438059553548 5.0
4.067295024522238 4.5
4.097268471298447 4.0
4.03350857645866 4.0
3.9625096019297885 4.0
3.464190200018431 3.0
4.008512515899159 4.0
3.9169654392861326 4.0
4.178754996532976 4.0
3.976738414137356 4.0
4.008512515899159 4.0
3.926497100406383 3.5
4.313285461007795 4.0
4.805366879449307 4.5
4.057399679532654 4.0
3.512389773998823 3.5
4.657006281591967 4.5
3.937340057246093 4.0
3.826189766609468 4.0
4.1320343835397075 4.0
4.006567054292077 4.0
4.281407455154085 4.